<a href="https://colab.research.google.com/github/roshnaShaji/telco-customer-churn-analysis/blob/main/Customer_churn_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [ ]:
import pandas as pd
import sqlite3

# Load the CSV
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Connect to SQLite (in memory)
conn = sqlite3.connect(":memory:")

# Save the DataFrame as a SQL table
df.to_sql("telco", conn, index=False, if_exists="replace")

7043

In [ ]:
# Count customers by churn status
query = "SELECT Churn, COUNT(*) as Count FROM telco GROUP BY Churn"
pd.read_sql_query(query, conn)

,Churn,Count
0,No,5174
1,Yes,1869


In [ ]:
#Churn by Contract Type
query = """
SELECT Contract, Churn, COUNT(*) as Count
FROM telco
GROUP BY Contract, Churn
"""
pd.read_sql_query(query, conn)

,Contract,Churn,Count
0,Month-to-month,No,2220
1,Month-to-month,Yes,1655
2,One year,No,1307
3,One year,Yes,166
4,Two year,No,1647
5,Two year,Yes,48


In [ ]:
# Avg monthly charges by churn
query = """
SELECT Churn, ROUND(AVG(MonthlyCharges), 2) as AvgMonthlyCharges
FROM telco
GROUP BY Churn
"""
pd.read_sql_query(query, conn)

,Churn,AvgMonthlyCharges
0,No,61.27
1,Yes,74.44


In [ ]:
# churn by tenure range, Shows newer customers tend to churn more
# useful business insight.
query = """
SELECT
  CASE
    WHEN Tenure < 12 THEN '<1 year'
    WHEN Tenure BETWEEN 12 AND 24 THEN '1–2 years'
    WHEN Tenure BETWEEN 25 AND 48 THEN '2–4 years'
    ELSE '4+ years'
  END AS TenureGroup,
  Churn,
  COUNT(*) as Count
FROM telco
GROUP BY TenureGroup, Churn
ORDER BY TenureGroup
"""
pd.read_sql_query(query, conn)

,TenureGroup,Churn,Count
0,1–2 years,No,809
1,1–2 years,Yes,332
2,2–4 years,No,1269
3,2–4 years,Yes,325
4,4+ years,No,2026
5,4+ years,Yes,213
6,<1 year,No,1070
7,<1 year,Yes,999


In [ ]:
import pandas as pd
import sqlite3

# Load CSV into a DataFrame
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Fix column if needed
df.columns = df.columns.str.strip()

# Clean column: SeniorCitizen to Yes/No
df['SeniorCitizen'] = df['SeniorCitizen'].replace({1: 'Yes', 0: 'No'})

# Load into SQLite
conn = sqlite3.connect(':memory:')
df.to_sql('telco', conn, index=False, if_exists='replace')

# Query example
query = """
SELECT SeniorCitizen, COUNT(*) as Count
FROM telco
GROUP BY SeniorCitizen
"""
pd.read_sql_query(query, conn)

,SeniorCitizen,Count
0,No,5901
1,Yes,1142


In [ ]:
# Step 5: Run SQL query to see Electronic Check churn breakdown
query1 = """
SELECT PaymentMethod, Churn, COUNT(*) AS Count
FROM telco
WHERE PaymentMethod = 'Electronic check'
GROUP BY Churn;
"""

electronic_check_churn = pd.read_sql_query(query1, conn)
print("Electronic Check churn breakdown:")
print(electronic_check_churn)

# Step 6: Run SQL query to calculate churn rate for Electronic Check
query2 = """
SELECT
  ROUND(
    100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2
  ) AS ChurnRate_ElectronicCheck
FROM telco
WHERE PaymentMethod = 'Electronic check';
"""

churn_rate = pd.read_sql_query(query2, conn)
print("\nChurn Rate for Electronic Check users:")
print(churn_rate)

Electronic Check churn breakdown:
      PaymentMethod Churn  Count
0  Electronic check    No   1294
1  Electronic check   Yes   1071

Churn Rate for Electronic Check users:
   ChurnRate_ElectronicCheck
0                      45.29
